In [1]:
import pandas as pd
import folium

In [6]:
df = pd.read_csv(r'Data\HBR.csv', header=0, sep=';', decimal=',')

# Sintaxe do google API:

In [7]:
dict_1 = {}
pontos=[]
for j in df['Rota'].unique():
    df_aux = df[df['Rota']==j]
    for _, row in df_aux.iterrows():
         pontos.append({'lat': row['Lat'],
                     'lon': row['Long'],
                     'title': row['Endereço']})
    dict_1[j] = pontos
    pontos = []

# Usando Folium

In [10]:
def split_df_rotas(df):
    # lista com número das rotas
    rotas_unique = df['Rota'].unique()
    # cria lista para receber os dfs com pontos de cada rota como itens
    df_rotas = ["" for x in range(len(rotas_unique)+1)]
    # preenche a lista com os dfs separados por rota
    for j in range(len(rotas_unique)+1):
        df_rotas[j] = df[df['Rota']==j]
    return df_rotas

In [11]:
df_rotas = split_df_rotas(df)

In [12]:
def plota_mapa(m, df_rotas, rota, cor):
    # reseta a var coord_anterior para não plotar a rota do último ponto de volta ao primeiro
    coord_anterior = 0

    for i, row in df_rotas[rota].iterrows():
        # plota os pontos de entrega
        folium.Marker([row['Lat'],row['Long']],
                      popup="Ponto " + str(i)+ ": " +str([row['Lat'],row['Long']]),
                      icon=folium.Icon(color=cor)).add_to(m)
        # plota as rotas
        try:
            folium.vector_layers.PolyLine([(row['Lat'],row['Long']),coord_anterior], color=cor).add_to(m)
        except:
            pass
        if coord_anterior == 0:
            folium.CircleMarker([row['Lat'],row['Long']],
                            color="green",
                            fill=True,
                            radius = 20,
                            popup="Ponto inicial").add_to(m)
        coord_anterior = (row['Lat'],row['Long'])
    return m

In [14]:
m = folium.Map(location=[df['Lat'].mean(),df['Long'].mean()], zoom_start=11)

plota_mapa(m, df_rotas,5,'red')
plota_mapa(m, df_rotas,11,'black')